### A full business solution: BUSINESS CHALLENGE:
Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

In [40]:
## required imports
import os
import json
from dotenv import load_dotenv
from web_scraper import fetch_website_contents, fetch_website_links
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [16]:
openai = OpenAI()
MODEL = 'gpt-5-nano'

In [6]:
# get all links
links = fetch_website_links("https://www.edwarddonner.com")
print(f"Found {len(links)} links")
print(f"Links: {links}")

Found 28 links
Links: ['https://edwarddonner.com/', 'https://edwarddonner.com/connect-four/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2025/11/11/ai-live-event/', 'https://edwarddonner.com/2025/11/11/ai-live-event/', 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/', 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/', 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/', 'https://edwarddonner.com/2025/05/28/connecting-

Here is this list of links we might not have any use of all the links: They can be relative links, they can be unrelated links etc. 

These unrelated links can be omitted from process, this can be done through an LLM call - 

It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about". Let us use `One Shot Prompting` technique to solve this problem:

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!


In [ ]:
# create prompt - using one shot prompting style

link_system_prompt = """
You are provided with a list of links found on a website.
You are able to decide which of these links are most relevant to be used in the company brochure,
such as about page, contact page, careers page, etc.

You should respond in a JSON format as follows in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "contact page", "url": "https://another.full.url/contact"},
    ]
}
"""

In [25]:
# user prompt

def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links found on the website: {url}
Decide which of these links are most relevant to be used in the company brochure,
repond with a full https url in JSON format.
Do not include terms of service, privacy policy, email links.

Links (some might have relative links):
"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)

    return user_prompt

In [19]:
print(get_links_user_prompt("https://www.edwarddonner.com"))


Here is the list of links found on the website: https://www.edwarddonner.com
Decide which of these links are most relevant to be used in the company brochure,
repond with a full https url in JSON format, provide maximum possible links.
Do not include terms of service, privacy policy, email links.

Links (some might have relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-g

In [20]:
def select_relevant_links(url):

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)},
        ],
        response_format={"type": "json_object"}
    )

    result = response.choices[0].message.content

    links = json.loads(result)

    return links

In [21]:
select_relevant_links("https://www.edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog index', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'brand page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'press release',
   'url': 'https://www.prnewswire.com/news-releases

In [22]:
select_relevant_links("https://www.health-chain.io")

{'links': [{'type': 'home page', 'url': 'https://health-chain.io/'},
  {'type': 'about page', 'url': 'https://health-chain.io/about-us/'},
  {'type': 'careers page', 'url': 'https://health-chain.io/careers/'},
  {'type': 'contact page', 'url': 'https://health-chain.io/contact-us/'},
  {'type': 'blog page', 'url': 'https://health-chain.io/blogs/'},
  {'type': 'case studies page',
   'url': 'https://health-chain.io/case-studies/'},
  {'type': 'webinars page', 'url': 'https://health-chain.io/webinars/'},
  {'type': 'press releases page',
   'url': 'https://health-chain.io/press-release/'},
  {'type': 'ebooks page', 'url': 'https://health-chain.io/ebooks/'},
  {'type': 'documentation page', 'url': 'https://docs.health-chain.io/'},
  {'type': 'payers page', 'url': 'https://health-chain.io/payers/'},
  {'type': 'solution page',
   'url': 'https://health-chain.io/cms-interoperability/'},
  {'type': 'solution page',
   'url': 'https://health-chain.io/data-management-quality/'},
  {'type': 'sol

### Make a brochure

In [32]:
def find_pages_and_relevant_links(url):
    print(f"Making calls to fetch to {MODEL}...")
    content = fetch_website_contents(url)
    relevent_links = select_relevant_links(url)

    print(f"Found {len(relevent_links['links'])} relevant links")

    result = f"## Landing Page\n\n{content}\n\n## Relevant Links\n\n"
    for link in relevent_links['links']:
        result += f"### Link: {link['type']}]\n"
        result += fetch_website_contents(link['url'])
        result += "\n"
        
    return result

In [27]:
print(find_pages_and_relevant_links("https://www.huggingface.co"))

## Landing Page

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanVideo-1.5
Updated
1 day ago
•
243
•
465
facebook/sam3
Updated
1 day ago
•
50.4k
•
458
WeiboAI/VibeThinker-1.5B
Updated
7 days ago
•
14k
•
435
moonshotai/Kimi-K2-Thinking
Updated
14 days ago
•
218k
•
1.35k
Supertone/supertonic
Updated
about 13 hours ago
•
594
•
150
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.18k
Qwen Image Edit Camera Control
🎬
Featured
1.18k
Fast 4 step inference with Qwen Image Edit 2509
Running
on
CPU Upgrade
Featured
2.36k
The Smol Training Playbook
📚
Featured
2.36k
The secrets to building world-class LLMs
Running
on
Zero
Featured
216
Depth Anything 3
🏢
Featured
216
Generate depth map

In [46]:
# brouchure_system_prompt = """
# You are an assistant that analyzes several relevant pages from the company's website.
# and create a short brochure for the company for prospective customers, investors and recruits.

# Respond in markdown format, without code blocks.
# Include details of company culture, current customers, and careers/jobs if you have the information.
# """

#  update the system prompt to be funny, engaging, witty and with a sense of humor

brouchure_system_prompt = """
You are an assistant that analyzes several relevant pages from the company's website.
and create a short, funny, engaging, witty and with a sense of humor brochure for the company for prospective customers, investors and recruits.

Respond in markdown format, without code blocks.
Include details of company culture, current customers, and careers/jobs if you have the information.
"""

In [29]:
def get_brouchure_user_promot(company_name, url):
    user_prompt = f"""
You are looking at a company called {company_name}
Here are the contents of its landing page and other relevant pages,
use this information to create a short brochure for the company, in markdown format without code blocks.
"""

    user_prompt += find_pages_and_relevant_links(url)
    user_prompt = user_prompt[:5000] # truncate to 5k tokens

    return user_prompt

In [30]:
get_brouchure_user_promot("Hugging Face", "https://www.huggingface.co")

'\nYou are looking at a company called Hugging Face\nHere are the contents of its landing page and other relevant pages,\nuse this information to create a short brochure for the company, in markdown format without code blocks.\n## Landing Page\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanVideo-1.5\nUpdated\n1 day ago\n•\n243\n•\n466\nfacebook/sam3\nUpdated\n1 day ago\n•\n50.4k\n•\n459\nWeiboAI/VibeThinker-1.5B\nUpdated\n7 days ago\n•\n14k\n•\n435\nmoonshotai/Kimi-K2-Thinking\nUpdated\n14 days ago\n•\n218k\n•\n1.35k\nSupertone/supertonic\nUpdated\nabout 14 hours ago\n•\n594\n•\n151\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP\nFeatured\n1.18k\nQwen 

In [31]:
def create_brouchure(company_name, url):

    response = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role": "system", "content": brouchure_system_prompt},
            {"role": "user", "content": get_brouchure_user_promot(company_name, url)},
        ],
    )

    result = response.choices[0].message.content

    display(Markdown(result))  # display the result

In [33]:
create_brouchure("Hugging Face", "https://www.huggingface.co")

Making calls to fetch to gpt-5-nano...
Found 5 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the **collaboration platform for the machine learning community**, helping to build the future of AI. It is the central hub where machine learning engineers, scientists, and AI enthusiasts can explore, share, and collaborate on models, datasets, applications, and research. Hugging Face empowers the next generation with open-source tools and a vibrant, fast-growing community.

---

## Our Platform

- **Models**: Access and contribute to over **1 million models** spanning text, image, video, audio, and 3D modalities.
- **Datasets**: Browse and share **250k+ datasets** to drive training and evaluation of AI models.
- **Spaces**: Create, run, and showcase live AI applications in a collaborative environment.
- **Community**: Join a global network of AI researchers, developers, and users to discuss, learn, and collaborate.
- **Open Source Stack**: Benefit from free, open-source ML libraries and tools designed for faster, collaborative AI development.

Build your ML portfolio, share your work, and engage with cutting-edge AI projects.

---

## Enterprise Solutions

Scale your AI initiatives securely and efficiently with Hugging Face's Enterprise Hub, offering:  

- **Team & Enterprise Plans**: Starting at $20/user/month with customizable contracts.
- **Enterprise-grade Security**: Single Sign-On (SSO), audit logs, granular access controls.
- **Advanced Compute**: Options like ZeroGPU for enhanced scalability and faster inference.
- **Private Storage & Dataset Viewer**: Collaboration-friendly private datasets and ample storage space.
- **Comprehensive Analytics**: Track usage and optimize team productivity.
- **Token and User Management**: Centralized control over organization access.

Harness the power of machine learning at scale with peace of mind.

---

## Company Culture

Hugging Face fosters an **open, ethical AI future** through open source and community-driven development. The culture is centered around:

- **Collaboration**: Encouraging knowledge-sharing and community contribution.
- **Innovation**: Supporting cutting-edge research and technology exploration.
- **Transparency**: Open-source ethos and ethical AI practices.
- **Inclusion**: Building a welcoming and diverse AI ecosystem.
- **Empowerment**: Enabling ML engineers and scientists to grow and showcase their expertise.

---

## Our Customers & Community

- Developers, researchers, and organizations worldwide use Hugging Face as their primary machine learning platform.
- Trusted by leading AI teams at **Facebook, Tencent, Nvidia**, and many startups.
- A vibrant ecosystem with tens of thousands of models, datasets, and applications continuously updated.
- Active community participation through GitHub, Discord, Twitter, and forums.

---

## Careers at Hugging Face

Join the company revolutionizing machine learning collaboration! Hugging Face offers exciting career opportunities for engineers, researchers, data scientists, and community managers. Working here means:

- Driving the future of AI through open-source innovation.
- Collaborating with top talent in a supportive and inspiring environment.
- Contributing to highly impactful projects with a global reach.

Explore current openings on the [Hugging Face Careers Page](https://huggingface.co/jobs).

---

## Get Involved

- Visit [huggingface.co](https://huggingface.co) to create a free account and start exploring.
- Join the conversation on GitHub, Discord, and Twitter.
- Build and host your own machine learning models and applications.
- Collaborate with thousands in the AI community working towards shared progress.

Together, let's build the future of AI.

---

**Hugging Face** – The AI community building the future.  
*Create. Collaborate. Innovate.*

### Streaming animation

In [44]:
def create_brouchure_stream(company_name, url):

    stream_response = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role": "system", "content": brouchure_system_prompt},
            {"role": "user", "content": get_brouchure_user_promot(company_name, url)},
        ],
        stream=True
    )

    chunk_text = ""
    display_handler = display(Markdown(""), display_id=True)
    for chunk in stream_response:
        chunk_text += chunk.choices[0].delta.content or ""
        update_display(Markdown(chunk_text), display_id=display_handler.display_id)

    

In [45]:
create_brouchure_stream("Hugging Face", "https://www.huggingface.co")

Making calls to fetch to gpt-5-nano...
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face - The AI Community Building the Future**

Hugging Face is the premier platform where the global machine learning community collaborates on creating and sharing models, datasets, and AI applications. It serves as the home of machine learning innovation — enabling users to create, discover, and collaborate better and faster.

Explore a vast ecosystem including over **1 million models**, **400,000+ applications**, and **250,000+ datasets** across various AI modalities: text, image, video, audio, and even 3D data.

Join a thriving community dedicated to open-source AI, sharing breakthroughs, and democratizing machine learning tools for developers, researchers, and enterprises alike.

---

## Key Features & Offerings

- **Models Hub:** Discover and share cutting-edge ML models, including trending ones like facebook/sam3 and tencent/HunyuanVideo-1.5.
- **Datasets:** Access a diverse and growing repository of datasets for various AI disciplines.
- **Spaces:** Host and deploy ML apps in an easy-to-use collaborative environment with free and paid compute options.
- **Collaboration Tools:** Git-based collaboration designed for version control and community input.
- **AI Modalities:** Support for text, image, video, audio, and 3D.
- **Build Your Portfolio:** Share your ML work and create a professional profile recognized by peers and employers.

---

## Company Culture

- **Community-Driven:** Hugging Face is committed to fostering an open-source culture where collaboration and knowledge-sharing underpin their innovation.
- **Inclusive & Supportive:** From beginners building their ML profiles to leading researchers, the platform supports all levels of expertise.
- **Innovation at Scale:** Encourages rapid experimentation with cutting-edge AI tools and hardware acceleration.
- **Education & Growth:** Provides access to resources like documentation, learning by playing with models and datasets, and blog publishing on user profiles.
- **Enterprise Ready:** Combines open collaboration with enterprise-grade security, compliance, and personalized support.

---

## Customers & Users

Hugging Face serves a wide range of users including:

- AI researchers and data scientists pushing the boundaries of ML.
- Developers creating real-world AI applications.
- Organizations transforming their business with AI through **Enterprise and Team plans**.
- Educational institutions advancing AI learning.
- Community contributors from around the world sharing knowledge and tools.

---

## Plans & Pricing

### Free Plan - Hugging Face Hub  
- Unlimited public models, datasets, and apps.  
- Access to community resources and tools.

### PRO Account - $9/month  
- Expanded private storage and inference credits.  
- Priority queue and enhanced hosting for Spaces.  
- Ability to publish blog articles and enhanced dataset viewer.

### Team Plan - $20/user/month  
- Single Sign-On (SSO) and SAML support.  
- Granular access controls and audit logs.  
- Centralized token control and usage analytics.  
- Advanced compute options for Spaces.  
- All members receive PRO benefits.

### Enterprise Plan - Starting at $50/user/month  
- Customized onboarding and enterprise features.  
- Highest storage, bandwidth and API rate limits.  
- Legal, compliance, and personalized support.  
- Flexible contracts with dedicated account management.

---

## Careers

Hugging Face is actively growing its team. Current openings span roles focused on:

- Machine Learning Research and Engineering  
- Software Development  
- Product Management  
- Community and Developer Relations  
- Enterprise Solutions and Support

Join a global team passionate about democratizing AI and revolutionizing the future of machine learning!

Check the [Hugging Face Careers Page](https://huggingface.co/careers) to explore current job opportunities.

---

## Get Started

- Explore AI models, datasets, and Spaces on the Hugging Face platform.  
- Build and share your own AI applications and profiles.  
- Join the open-source movement with an inclusive and supportive community.  
- Scale your AI initiatives with enterprise-grade security and support.

**Visit:** [huggingface.co](https://huggingface.co)

**Sign Up Today and Build the Future of AI Together!**

---

*Hugging Face – Where AI Innovation Meets Community*

In [47]:
create_brouchure_stream("Hugging Face", "https://www.huggingface.co")

Making calls to fetch to gpt-5-nano...
Found 16 relevant links


# Hugging Face: The AI Playground Where Tomorrow Gets Built Today

---

## Who Are We, You Ask?

Welcome to **Hugging Face** — the AI community that isn’t just *building the future*, we're hugging it tight and making it smarter! Think of us as the coolest coffeehouse for machine learning models, datasets, and AI apps, buzzing with engineers, scientists, and curious minds who love open-source and open minds.

---

## What’s Cooking at Hugging Face?

### 1 Million+ Models & Counting  
From video-editing AI that directs your scenes (tencent/HunyuanVideo-1.5) to creative buddies like Dream-wan2-2-faster-Pro that spin magic from images, our model library is a smorgasbord of AI brilliance.

### 250k+ Datasets  
If data is the new oil, consider us the refinery that fuels innovation. Autonomous vehicles, synthetic lives, and everything in-between — all accessible and ready for your next ML masterpiece.

### Spaces & Apps  
Hosting and collaborating on apps? Check. Demoing next-gen image editors or depth mappers? Double-check. Build, share, wow! The AI stage is yours.

---

## Why Hugging Face? Because We’re Not Your Average AI Crowd

- **Collaboration Nation** — Unlimited public models and datasets! Share, explore, and dip into a community so friendly, it feels like your café tech tribe.
- **Open & Ethical AI** — We’re building next-gen tech without the shady side. Transparency and ethics baked right in.
- **Fast & Furious** — Our open-source stack will have your projects speeding toward breakthrough faster than you can say "ZeroGPU mode."
- **All Modalities Welcome** — Text, image, video, audio, or 3D — we don't discriminate. AI loves variety, and so do we.

---

## Meet the Crew (That Could Be You!)

Join the **heart of the AI revolution**, where the science team is as talented as a rock band with APIs instead of guitars. Careers at Hugging Face mean:

- Being part of a fast-growing, high-impact community  
- Making open-source magic that reaches millions  
- Working in a culture that values curiosity, collaboration, and of course — a bit of wit  
- Open positions ready for trailblazers who want to shape AI’s future

Interested? Peek at our [Careers Page](https://huggingface.co/jobs) — your dream job is only a click away.

---

## The VIP Pass: Pricing Plans That Suit Your Groove

- **Pro Account:** $9/month — Supercharge your personal AI adventures. More storage, priority queues, and flashy PRO badges to put on your profile.  
- **Team Plan:** $20/user/month — Power up your squad with SSO, granular controls, audit logs, and sharing superpowers.  
- **Enterprise:** Custom pricing — Tailored solutions, legal peace of mind, and personalized support. Because big AI dreams need big support.

---

## Join the Hug!

Ready to explore what 1M+ models, 250k+ datasets, and a global community buzzing with AI excitement can do for you? Whether you’re a hobbyist, a groundbreaking researcher, or an enterprise visionary, Hugging Face is the home where AI grows, learns, and occasionally cracks a joke or two.

> **Hugging Face — Where AI Gets a Warm Welcome and a Brainy Boost.**

✨ *Sign up today and join the AI hug revolution!* ✨

---

### Stay Connected  
- [GitHub](https://github.com/huggingface) — Code, contribute, collaborate!  
- [Twitter](https://twitter.com/huggingface) — Catch our latest AI capers  
- [Discord](https://discord.gg/huggingface) — Community chat & cheer  

---

*Powered by curiosity, a dash of code, and a lot of good vibes.*